In [1]:
import pandas as pd
import numpy as np
import datetime
import string
import os
from matplotlib import pyplot as plt
from pathlib import Path

In [2]:
DATA_PATH = '/Users/vuhoangnguyen/Research/'
RAW_DATA = os.path.join(DATA_PATH, 'raw_data.csv')
OUT_DATA = os.path.join(DATA_PATH, 'out.csv')

In [3]:
dataset = pd.read_csv(RAW_DATA, keep_default_na=False)

In [4]:
dataset = dataset.rename(columns={"GIOI.x": "Female"})
s = dataset['Female'].to_numpy()

for i in range(len(s)):
    if s[i] == 'Nam' or s[i] == 'M':
        dataset.at[i,'Female'] = 0
    elif s[i] == 'Nu' or s[i] == 'F':
        dataset.at[i,'Female'] = 1

In [5]:
s = dataset['Female'].to_numpy()
male = 0
female = 0
total = dataset.shape[0]
for i in s:
    if i == 0:
        male += 1
    elif i == 1:
        female += 1
unknown = total - male - female
gender = pd.DataFrame([[total,male,female,unknown]], columns=['Total','Male','Female','Unknown'])

In [6]:
filepath = Path(os.path.join(DATA_PATH, 'gender.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
gender.to_csv(filepath, index=False)

In [7]:
dataset = dataset.drop(columns=['STT', 'ID', 'Name', 'SODIENTHOAI', 'Fisrt of Department',
                                'Department of out hospital', 'Department of test'])

In [8]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr',
              'May', 'Jun', 'Jul', 'Aug',
              'Sep', 'Oct', 'Nov', 'Dec']

def to_datetime(column_name):
    s = dataset[column_name].to_numpy(dtype = 'str')
    for i in range(len(s)):
        if s[i] == 'nan':
            continue
        if '-' in s[i]:
            date = s[i].split(sep='-')
        elif '/' in s[i]:
            date = s[i].split(sep='/')
        for j in range(len(month_name)):
            if date[1] == month_name[j]:
                date[1] = j+1
        date = [int(x) for x in date]
        if date[0] > 1000: # date[0] is year
            x = datetime.date(date[0],date[1],date[2])
        else:
            if date[2] < 1000: # date[2] is year
                date[2] += 2000
            x = datetime.date(date[2],date[1],date[0])
        dataset.at[i,column_name] = x

dataset = dataset.rename(columns={"NGAYSINH.x":"dob"})
to_datetime('dob')
dataset = dataset.rename(columns={"date of admission":"admission_date"})
to_datetime('admission_date')

In [9]:
dataset = dataset.rename(columns={"date of onset":"onset_date"})
onset_date_str = dataset['onset_date'].to_numpy(dtype = 'str')

for i in range(len(onset_date_str)):
    current_date = onset_date_str[i]
    if current_date == 'nan':
        continue
    if '-' in current_date:
        date = current_date.split(sep='-')
    if '/' in current_date:
        date = current_date.split(sep='/')
    for j in range(len(month_name)):
        if date[1] == month_name[j]:
            date[1] = j+1
    date = [int(x) for x in date]
    if date[2] < 1000: # date[2] is year
        date[2] += 2000
    x = datetime.date(date[2],date[1],date[0])
    dataset.at[i,'onset_date'] = x

In [10]:
# correct row 1085 (21/7/209) to 21/7/2019

for i in range(dataset.shape[0]):
    d = dataset.at[i,'onset_date'].day
    m = dataset.at[i,'onset_date'].month
    y = dataset.at[i,'onset_date'].year
    if y not in [2017,2018,2019,2014]:
        dataset.at[i,'onset_date'] = datetime.date(2019, m, d)

In [11]:
dataset.insert(7, "age_at_admission", 0)

In [12]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'dob']
    y = dataset.at[i,'admission_date']
    if pd.isna(dataset.at[i,'dob']) == False and pd.isna(dataset.at[i,'admission_date']) == False:
        z = y - x
        dataset.at[i,"age_at_admission"] = z.days // 30

In [13]:
dataset = dataset.rename(columns={"Distric":"district"})
dis = dataset['district'].to_numpy(dtype='str')

wrong_dis = ["Huyen V_ Quang","ph_ c_","Huyen Hi_p H","Kim B_ng","Huyen H__ng Kh?","Nam Tr_c","Th__ng Xu","T__ng D__ng","Huyen C_ Kuin",
"B_m S_n","Y?n Khnh","Bnh Xuy?n","Huyen Kim __ng","Xun Tr__ng","Tam D__ng","Huyen Qu_nh L_u","Thi Xa Thai Ho?a","Thi Xa Thai Ho`a",
"TP B_c Giang","L__ng S_n","Y?n L_c","Ung B","Huyen Nng C_ng","Kim Thnh","Huyen Di_n Ch","T_ k_","Gia Vi_n","Giao Th_y","TP. Thi Nguy?n",
"L_p Th_ch","L_c Nam","Huyen T_ K_","tp L_ng S_n","tp. Ninh Bnh","Vi_t Tr","Huyen C_m Kh?","Phc Thnh","B_c Quang","Huyen Thanh Ch__ng",
"H_ H","Huyen Bnh Giang","Mai S_n","Huyen Tam Nâ¢ng","Th_y Nguy?n","Tp Thi Bnh","Ngh_a H_ng","_inh H","Thnh ph_ H_i D__ng","TP. Ninh Bnh",
"Ch Linh","TP. Nam __nh","TP.H_ng Y?n","Kinh M_n","Thanh Mi_n","H_ng Nguy?n","Nam Sch","Huyen Ngh_a _","H_u L_c","Nam c__ng","Quang S_n",
"Phc Y?n","Gia Bnh","Huyen _ng H_ng","Y?n M_","ån Thi","V_nh t__ng","Huyen V_n L","L_c H","__ S_n","H Phong","__ng T"]
correction = ["Huyen Vu Quang","Phu Cu","Huyen Hiep Hoa","Kim Bang","Huyen Huong Khe","Nam Truc","Thuong Xuan","Tuong Duong","Huyen Cu Kuin",
"Bim Son","Yen Khanh","Binh Xuyen","Huyen Kim Dong","Xuan Truong","Tam Duong","Huyen Quynh Luu","Thi Xa Thai Hoa","Thi Xa Thai Hoa",
"TP Bac Giang","Luong Son","Yen Lac","Uong Bi","Huyen Nong Cong","Kim Thanh","Huyen Dien Chau","Tu ky","Gia Vien","Giao Thuy","TP Thai Nguyen",
"Lap Thach","Luc Nam","Huyen Tu Ky","TP Lang Son","TP Ninh Binh","Viet Tri","Huyen Cam Khe","Phuc Thanh","Bac Quang","Huyen Thanh Chuong",
"Ha Hoa","Huyen Binh Giang","Mai Son","Huyen Tam Nong","Thuy Nguyen","TP Thai Binh","Nghia Hung","Dinh Hoa","TP Hai Duong","TP Ninh Binh",
"Chi Linh","TP Nam Dinh","TP Hung Yen","Kinh Mon","Thanh Mien","Hung Nguyen","Nam Sach","Huyen Nghia Dan","Hau Loc","Nam Cuong","Quang Son",
"Phuc Yen","Gia Binh","Huyen Dong Hung","Yen My","An Thi","Vinh Tuong","Huyen Van Lam","Loc Ha","Do Son","Hong Phong","Dong Tam"]

for i in range(len(dis)):
    if dis[i] == 'nan':
        dataset.at[i,'district'] = ''
        continue
    for j in range(len(wrong_dis)):
        if dis[i] == wrong_dis[j]:
            dis[i] = correction[j]
            break
    word_dis = dis[i].split(sep = ' ')
    if word_dis[0] == 'Huy_n':
        word_dis[0] = 'Huyen'
    if word_dis[0] == 'TX':
        word_dis[0] = 'Thi Xa'
    new_dis = ' '.join(word_dis)
    dataset.at[i,'district'] = new_dis

In [14]:
dis = dataset['district'].to_numpy(dtype='str')

for i in range(len(dis)):
    if dis[i] == '':
        continue
    dis[i] = string.capwords(dis[i])
    dataset.at[i,'district'] = dis[i]

In [15]:
dataset = dataset.rename(columns={"Province": "province"})
pro = dataset['province'].to_numpy(dtype='str')

for i in range(len(dis)):
    if pro[i] == 'nan':
        continue
    pro[i] = string.capwords(pro[i])
    dataset.at[i,'province'] = pro[i]

In [16]:
dataset = dataset.rename(columns={"Community": "commune"})
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == 'nan':
        continue
    s = ""
    for j in comm[i]:
        if (j>='a' and j<='z') or (j>='A' and j<='Z') or j ==' ' or (j>='0' and j<='9'):
            s += j
    dataset.at[i,'commune'] = s

In [17]:
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == '':
        continue
    comm[i] = string.capwords(comm[i])
    dataset.at[i,'commune'] = comm[i]

In [18]:
dataset.insert(5,"complete_patient_address",np.nan)

In [19]:
for i in range(dataset.shape[0]):
    commune_i = dataset.at[i,'commune']
    district_i = dataset.at[i,'district']
    province_i = dataset.at[i,'province']
    address = []
    if commune_i != '':
        address.append(commune_i)
    if district_i != '':
        address.append(district_i)
    if province_i != '':
        address.append(province_i)
    full_address = ', '.join(address)
    dataset.at[i,'complete_patient_address'] = full_address

In [20]:
dataset = dataset.rename(columns = {"date of out hospital":"discharge_date"})
to_datetime('discharge_date')

In [21]:
dataset.insert(10,'duration_of_stay',np.nan)

In [22]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'admission_date']
    y = dataset.at[i,'discharge_date']
    if pd.isna(x) == False and pd.isna(y) == False:
        z = y - x
        dataset.at[i,"duration_of_stay"] = z.days

In [23]:
dataset = dataset.drop(columns = ['LOAIXN','Note of time stay in hospital'])

In [24]:
dataset = dataset.rename(columns = {'date of test':'test_date','date of result':'testresult_date'})
to_datetime('test_date')
to_datetime('testresult_date')

In [25]:
dataset = dataset.rename(columns = {'HAI':'healthcare_associated_infection','RSV':'respiratory_syncytical_virus','ADENO':'adenovirus',
'Pertussis':'pertussis','oxygen':'oxygen_cannula','cipap':'CPAP','ventilator':'conventional_mechanical_ventilation',
'HFO ventilator':'hfo_ventilation','ECMOY:AL':'ECMO'})

def YN_to_binary(column_name):
    s = dataset[column_name].to_numpy(dtype='str')
    for i in range(len(s)):
        if s[i] == '' or s[i] == 'nan':
            dataset.at[i,column_name] = 0
            continue
        ls = s[i].lower()
        if ls == 'yes':
            dataset.at[i,column_name] = 1
        elif ls == 'no' or ls == 'na':
            dataset.at[i,column_name] = 0

YN_to_binary('oxygen_cannula')
YN_to_binary('CPAP')
YN_to_binary('pertussis')
YN_to_binary('healthcare_associated_infection')
YN_to_binary('respiratory_syncytical_virus')
YN_to_binary('adenovirus')
YN_to_binary('conventional_mechanical_ventilation')
YN_to_binary('hfo_ventilation')
YN_to_binary('ECMO')

In [26]:
diseases = pd.DataFrame()
def diseases_data(column_name):
    s = dataset[column_name].to_numpy(dtype = 'int')
    y = 0
    n = 0
    for i in s:
        if i == 1:
            y += 1
        elif i == 0:
            n += 1
    #if y + n != len(s):
    #    print('nan exists')
    row = diseases.shape[1]
    diseases.insert(row,column_name,[n,y])
diseases_data('adenovirus')
diseases_data('pertussis')
diseases_data('healthcare_associated_infection')
diseases_data('respiratory_syncytical_virus')

In [27]:
from pathlib import Path  
filepath = Path(os.path.join(DATA_PATH, 'diseases.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
diseases.to_csv(filepath, index = True)

In [28]:
from datetime import date
def datetime_to_date(column_name):
    for i in range(dataset.shape[0]):
        x = dataset.at[i,column_name]
        y = pd.to_datetime(x)
        z = y.date()
        dataset.at[i,column_name] = z

datetime_to_date('dob')
datetime_to_date('admission_date')
datetime_to_date('discharge_date')
datetime_to_date('test_date')
datetime_to_date('testresult_date')

In [29]:
dataset = dataset.drop(columns = ['dialysis','Catheter','Onsetto out hospital','ghi chu nguon'])

In [30]:
dataset = dataset.rename(columns={'Code source': 'place_of_exposure'})

dataset['place_of_exposure'] = dataset['place_of_exposure'].str.lower()

for i in range(len(dataset['place_of_exposure'])):
    code = dataset.at[i, 'place_of_exposure']
    if code in ['suppected nch', 'definitely nch']:
        dataset.at[i, 'place_of_exposure'] = 'NHP'
    elif code in ['suppected other h', 'suppected  other h', 'definitely other h']:
        dataset.at[i, 'place_of_exposure'] = 'Another hospital'
    elif code in ['community']:
        continue

In [31]:
# Calculate number of patients in each place of exposure
in_nhp, in_another_hospital, in_community = 0, 0, 0
total = dataset.shape[0]

for i in dataset['place_of_exposure']:
    if i == 'NHP':
        in_nhp += 1
    elif i == 'Another hospital':
        in_another_hospital += 1
    elif i == 'community':
        in_community += 1
unknown = total - in_nhp - in_another_hospital - in_community
poe = pd.DataFrame([[total, in_nhp, in_another_hospital, in_community, unknown]], columns=['Total', 'NHP', 'Another hospital', 'Community', 'Unknown'])

In [32]:
from pathlib import Path  
filepath = Path(os.path.join(DATA_PATH, 'place_of_exposure.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
poe.to_csv(filepath, index=True)

In [33]:
dataset.insert(20,'complication-gastroentiritis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    if 'tieu chay' in x:
        dataset.at[i,'complication-gastroentiritis'] = 1

In [34]:
dataset.insert(21,'complication-middle-ear-infec',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    if 'viem tai' in x or 'viem tai' in y or 'viem tai' in z:
        dataset.at[i,'complication-middle-ear-infec'] = 1

In [35]:
dataset.insert(22,'complication-conjunctivitis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    if 'ket mac' in x or 'ket mac' in y or 'ket mac' in z:
        dataset.at[i,'complication-conjunctivitis'] = 1

In [36]:
dataset.insert(23,'complication-laryngitis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']  
    if 'thanh quan' in x or 'thanh quan' in y or 'thanh quan' in z or 'thanh quan' in t:
        dataset.at[i,'complication-laryngitis'] = 1

In [37]:
dataset.insert(24,'complication-pneumonia-bronchitis',0)
key_word = ['viem phoi', 'viem phe quan','phe quan']

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    for word in key_word:
        if word in x or word in y or word in z or word in t:
            dataset.at[i,'complication-pneumonia-bronchitis'] = 1  
            break

In [38]:
dataset.insert(25,'complication-febrile-seizures',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    if 'dong kinh' in x or 'dong kinh' in y or 'dong kinh' in z or 'dong kinh' in t:
        dataset.at[i,'complication-febrile-seizures'] = 1

In [39]:
dataset.insert(26,'complication-septic-shock-sepsis',0)
key_word = ['nhiem khuan huyet', 'soc nhiem khuan']

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    for word in key_word:
        if word in x or word in y or word in z or word in t:
            dataset.at[i,'complication-septic-shock-sepsis'] = 1

In [40]:
dataset.insert(27,'co-infection-influenza-a',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'cum a' in x:
            dataset.at[i,'co-infection-influenza-a'] = 1
            break

In [41]:
dataset.insert(28,'co-infection-influenza-b',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'cum b' in x:
            dataset.at[i,'co-infection-influenza-a'] = 1
            break

In [42]:
dataset.insert(29,'co-infection-streptococus-aerius',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'tu cau' in x:
            dataset.at[i,'co-infection-streptococus-aerius'] = 1
            break

In [43]:
dataset.insert(30,'co-infection-streptococus-pneumonia',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'phe cau' in x:
            dataset.at[i,'co-infection-streptococus-pneumonia'] = 1
            break

In [44]:
special_vaccine_status = ['','1 (12/2/2018)','1 (12/22/2018)','99','?a khoa qu?c t? H?i Phòng, ?i?u tr? ph?i','NA',
'd<U+1EE7> mui','kham bs lan anh','tiêm 15/9']
vaccine_value = ['NA',1,1,'NA','NA','NA',2,'NA',1]
vacc = dataset['Vaccination'].to_numpy(dtype=str)
value_occurance = {}
for i in range(dataset.shape[0]):
    for j in range(len(special_vaccine_status)):
        if vacc[i] == special_vaccine_status[j]:
            vacc[i] = vaccine_value[j]
            break
    dataset.at[i,'Vaccination'] = vacc[i]
    if vacc[i] in value_occurance:
        value_occurance[vacc[i]] += 1
    else:
        value_occurance[vacc[i]] = 1

item_list = sorted(list(value_occurance.items()))
vaccine_occurance = pd.DataFrame(item_list,columns=['Vaccine status','Occurance'])

filepath = Path(os.path.join(DATA_PATH, 'vaccine.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
vaccine_occurance.to_csv(filepath, index = True)

# calculate vaccine status of patient > 9 months old
value_occurance = {}
for i in range(dataset.shape[0]):
    if dataset.at[i,'age_at_admission'] >= 9:
        if 'Total' in value_occurance:
            value_occurance['Total'] += 1
        else:
            value_occurance["Total"] = 1
        if vacc[i] in value_occurance:
            value_occurance[vacc[i]] += 1
        else:
            value_occurance[vacc[i]] = 1

item_list = sorted(list(value_occurance.items()))
vaccine_occurance = pd.DataFrame(item_list,columns=['Vaccine status','Occurance'])

filepath = Path(os.path.join(DATA_PATH, 'vaccine status of patients more than 9 months old.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)  
vaccine_occurance.to_csv(filepath, index = True)

In [45]:
month_at_admission = dataset['age_at_admission'].to_numpy() // 30
max_month = np.max(month_at_admission)
val,bin = np.histogram(month_at_admission, bins=np.arange(max_month+2))

fig, ax = plt.subplots()

# Set title, add labels
ax.set_title("Histogram of age at admission")
ax.set_xlabel('Age (month)')
ax.set_ylabel('Number of patients')

# can add log = 'True' for better visualization
ax.hist(month_at_admission,bins = np.arange(max_month+2),align='left',edgecolor='black',label='Age at admission')
rects = ax.patches
  
for rect, label in zip(rects, val):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width() / 2, height+0.01, label, ha='center', va='bottom')

# remove this line to see the box plot
plt.close()

In [46]:
def print_stats(array,array_name):
    min = np.min(array)
    max = np.max(array)
    mean = np.mean(array)
    median = np.median(array)
    first_quartile = np.percentile(array, 25)
    third_quartile = np.percentile(array, 75)

    print(f'Min {array_name} = {min}')
    print(f'Max {array_name} = {max}')
    print(f'Mean {array_name} = {mean}')
    print(f'Median {array_name} = {median}')
    print(f'First quartile {array_name} = {first_quartile}')
    print(f'Third quartile {array_name} = {third_quartile}')

In [47]:
duration_of_stay = dataset['duration_of_stay'].to_numpy()

# don't need box plots yet
#plt.boxplot(duration_of_stay,showmeans=True,vert=False)
#plt.show()

# remove this comment to see the stats
#print_stats(duration_of_stay,'duration of stay')

duration_group = [0,3,8,15]
occurance_of_duraion_group = [0] * 4

for i in duration_of_stay:
    for j in range(3,-1,-1):
        if i >= duration_group[j]:
            occurance_of_duraion_group[j] += 1
            break

stay_duration = pd.DataFrame([occurance_of_duraion_group], columns=['< 3','3 - 7','8 - 14','> 14'])

In [48]:
filepath = Path(os.path.join(DATA_PATH, 'duration_of_stay.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
stay_duration.to_csv(filepath, index=False)

In [49]:
test_date_to_test_result = dataset['testresult_date'].to_numpy() - dataset['test_date'].to_numpy()
for i in range(len(test_date_to_test_result)):
    test_date_to_test_result[i] = test_date_to_test_result[i].days

# remove this comment to see the stats
#print_stats(test_date_to_test_result,'test duration')

plt.boxplot(test_date_to_test_result,showmeans=True,vert=False)

# remove this command to see the box plot
plt.close()

day_group = [0,1,2,3]
occurance_of_day_group = [0] * 4

for i in test_date_to_test_result:
    for j in range(3,-1,-1):
        if i >= day_group[j]:
            occurance_of_day_group[j] += 1
            break

test_duration = pd.DataFrame([occurance_of_day_group], columns=['0','1','2','> 2'])

In [50]:
filepath = Path(os.path.join(DATA_PATH, 'duration_of_test.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
test_duration.to_csv(filepath, index=False)

In [51]:
onset_date_to_admission_date = dataset['admission_date'].to_numpy() - dataset['onset_date'].to_numpy()
for i in range(len(onset_date_to_admission_date)):
    onset_date_to_admission_date[i] = onset_date_to_admission_date[i].days

# remove this comment to see the stats
#print_stats(onset_date_to_admission_date,'onset to admission')

In [52]:
onset_date_to_test_date = dataset['test_date'].to_numpy() - dataset['onset_date'].to_numpy()
for i in range(len(onset_date_to_test_date)):
    onset_date_to_test_date[i] = onset_date_to_test_date[i].days

# remove this comment to see the stats
#print_stats(onset_date_to_test_date,'onset to test')

In [53]:
admission = dataset['admission_date'].to_numpy()
admission_year = [x.year for x in admission]
occurance_of_year = {}

for i in admission_year:
    if i not in occurance_of_year:
        occurance_of_year[i] = 1
    else:
        occurance_of_year[i] += 1

x = list(occurance_of_year.items())
x.sort()
occurance_value = [current_year[1] for current_year in x]
year_occurance = pd.DataFrame([occurance_value],columns = ['2017','2018','2019'])

week_occurance = {}
for i in admission:
    current_week = i.strftime("%V")
    current_year = i.year
    current_time = str(current_year) + '-' + current_week
    if current_time not in week_occurance:
        week_occurance[current_time] = 1
    else:
        week_occurance[current_time] += 1

dict_list = [(k, v) for k, v in week_occurance.items()]
dict_list.sort()
year_week = [current_week[0] for current_week in dict_list]
week_occurance = [current_week[1] for current_week in dict_list]

plt.figure(figsize=(20,11))
plt.bar(year_week, week_occurance, color='g')
plt.xticks(rotation=90)

# comment this line to see the bar chart
plt.close()

plt.show()

In [54]:
filepath = Path(os.path.join(DATA_PATH, 'yearly_patient.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
year_occurance.to_csv(filepath, index=False)

In [55]:
# add new columns to Dataframe diseases 
column_name = ['complication-gastroentiritis','complication-middle-ear-infec','complication-conjunctivitis','complication-laryngitis','complication-pneumonia-bronchitis',
'complication-febrile-seizures','complication-septic-shock-sepsis','co-infection-influenza-a','co-infection-influenza-b','co-infection-streptococus-aerius',
'co-infection-streptococus-pneumonia']

total = dataset.shape[0]
for col in column_name:
    row = diseases.shape[1]
    y = dataset[col].sum()
    diseases.insert(row,col,[total-y,y])

In [56]:
dataset.insert(39,'highest_ventilation_mode',0)

order = ['oxygen_cannula','CPAP','conventional_mechanical_ventilation','hfo_ventilation','ECMO']
patient_per_ventilation_mode = [0] * 5

for i in range(len(dataset)):
    for j in range(len(order)-1,-1,-1):
        if dataset.loc[i,order[j]] == 1:
            dataset.loc[i,'highest_ventilation_mode'] = order[j]
            patient_per_ventilation_mode[j] += 1
            break

ventilation_mode = pd.DataFrame([patient_per_ventilation_mode],columns = order)

In [57]:
filepath = Path(os.path.join(DATA_PATH, 'ventilation_mode.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
ventilation_mode.to_csv(filepath, index=False)

In [58]:
import folium
from folium import plugins

m = folium.Map(location=[18.975670, 105.608640])
# m

In [59]:
stage_of_discharge_value = ['BN khong nhap vien','Bo ve','Chuyen vien','Ra vi<U+1EC7>n','Ra vien',
'Tu vong','Xin ve','chua ra vien','do giam','do, giam ','khoi','tu vong']
stage_of_discharge_new_value = ['khong nhap vien','bo ve','chuyen vien','ra vien','ra vien',
'tu vong','xin ve','chua ra vien','giam','giam','khoi','tu vong']

result_of_discharge_value = ['Chuyen  vien','Do/Giam','Khoi','Khong thay doi','Nang hon','Tu vong',
'chua ra vien','do giam','do, giam ','khoi','tu vong','Ð<U+1EE1>/Gi<U+1EA3>m']
result_of_discharge_new_value = ['chuyen vien','giam','khoi','khong thay doi','nang hon','tu vong',
'chua ra vien','giam','giam','khoi','tu vong','giam']

for i in range(len(dataset)):
    for j in range(len(stage_of_discharge_value)):
        if dataset.loc[i,'Stage of discharge'] == stage_of_discharge_value[j]:
            dataset.loc[i,'Stage of discharge'] = stage_of_discharge_new_value[j]
            break
    for j in range(len(result_of_discharge_value)):
        if dataset.loc[i,'result of discharge'] == result_of_discharge_value[j]:
            dataset.loc[i,'result of discharge'] = result_of_discharge_new_value[j]
            break

#stage_and_result = dataset[['Stage of discharge','result of discharge']]
#stage_and_result = stage_and_result.groupby(['Stage of discharge','result of discharge']).size().reset_index(name='counts')

dataset.insert(14,'outcome_died',0)

for i in range(len(dataset)):
    if dataset.loc[i,'result of discharge'] == 'tu vong':
        dataset.loc[i,'outcome_died'] = 1
    if dataset.loc[i,'Stage of discharge'] == 'tu vong':
        dataset.loc[i,'outcome_died'] = 1
    if dataset.loc[i,'Stage of discharge'] == 'xin ve' and dataset.loc[i,'result of discharge'] == 'nang hon':
        dataset.loc[i,'outcome_died'] = 1

column_name = ['dead','discharge','alive at hospital']
patient_result = [0] * 3
discharge_stage = ['ra vien','chuyen vien','xin ve']

for i in range(len(dataset)):
    if dataset.loc[i,'outcome_died'] == 1:
        patient_result[0] += 1
    elif dataset.loc[i,'Stage of discharge'] in discharge_stage:
        patient_result[1] += 1
    else:
        patient_result[2] += 1

dataset = dataset.drop(columns=['result of discharge','Stage of discharge'])

patient_result = pd.DataFrame([patient_result],columns = column_name)

In [60]:
filepath = Path(os.path.join(DATA_PATH, 'patient_result.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
patient_result.to_csv(filepath, index=False)

In [61]:
UPDATE_DATA = os.path.join(DATA_PATH, 'Duplicate.csv')
update_dataset = pd.read_csv(UPDATE_DATA, keep_default_na = True)
dataset.insert(6,'patient_home_coordinate',update_dataset['patient_home_coordinate'])
dataset.insert(7,'distance_to_nhp',update_dataset['distance_to_nhp'])

In [64]:
patient_location = dataset[['patient_home_coordinate','distance_to_nhp']].copy()
patient_location.insert(0,'index',range(0, len(patient_location)))
filepath = Path(os.path.join(DATA_PATH, 'patient_location.csv'))
filepath.parent.mkdir(parents=True, exist_ok=True)
patient_location.to_csv(filepath, index=False)

In [63]:
filepath = Path(OUT_DATA)
filepath.parent.mkdir(parents=True, exist_ok=True)
dataset.to_csv(filepath, index=False)